In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2",
                           dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
res = client.register(name="a", email="aa@b.org", password="c", password_verify="c")

In [4]:
ds_client = node.login(email="aa@b.org", password="c")

Logged into <test-domain-helm2: High side Domain> as <aa@b.org>


setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [5]:
dataset = sy.Dataset(
    name="My dataset",
    asset_list=[
        sy.Asset(
            name="input asset",
            data=ActionObject.from_obj([1, 2]),
            mock=ActionObject.from_obj([1, 2]),
        ),
    ]
)

In [6]:
# dataset.assets[0].mock.syft_action_data

# dataset.assets[0].data.syft_action_data

# dataset.assets[0].mock.__len__()

# dataset.assets[0].data.__len__()

In [7]:
dataset.assets[0].data.syft_action_data

[1, 2]

In [8]:
client.upload_dataset(dataset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.25it/s]

Uploading: input asset


SyftSuccess: Dataset uploaded to 'test-domain-helm2'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [9]:
asset = ds_client.datasets[0].assets[0]

In [10]:
# x = ActionObject.from_obj([1, 2])
# x_ptr = x.send(ds_client)

## Batch function

In [11]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [12]:
ds_client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [13]:
@syft_function_single_use(x=asset)
def process_all(domain, x):
    
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 3

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [14]:
ds_client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request 5147062631e1422f8a113120f153afa4 changes applied

In [15]:
# ds_client.code.process_all(x=asset, blocking=False)

In [17]:
mock =asset.mock

In [18]:
mock

[1, 2]

In [19]:
# "domain" in sig.parameters

In [20]:
from syft.node.worker import Worker

In [21]:
from syft.node.node import Node

In [22]:
# process_all
# process_all(x=[3,4])

In [23]:
# domain = Node.named(name="plan_building", reset=True, processes=0)

In [24]:
domain = sy.orchestra.launch(name="test",
                           dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=22221)
                           
temp_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [25]:
# handle = NodeHandle(
#     node_type=node_type_enum,
#     deployment_type=deployment_type_enum,
#     name=name,
#     python_node=worker,
#     node_side_type=node_side_type,
# )

In [26]:
ds_client.code.process_all

<function syft.client.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [29]:
client.launch_job

<bound method DomainClient.launch_job of <DomainClient: test-domain-helm2>>

In [30]:
# from hagrid.hagrid.orchestra

In [31]:
# handle.login(email="info@openmined.org", password="changethis")

In [32]:
filtered_kwargs = {"x": [3,4]}

In [33]:
filtered_kwargs["domain"] = temp_client

In [37]:
action_kwargs = {}
for k, v in filtered_kwargs.items():
    val = ActionObject.from_obj(v)
    action_kwargs[k] = val.send(temp_client)


In [38]:
process_all.local_function(**action_kwargs)

Launching jobs


AttributeError: 'SQLiteStorePartition' object has no attribute 'lock'

Idle worker expired: b'5C5B-F4BE'
Idle worker expired: b'D140-D0EF'
Idle worker expired: b'4A71-B0A6'
Idle worker expired: b'4999-97E5'
Idle worker expired: b'897B-20D9'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No s

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherl

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherl

...

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherl

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/queue/zmq_queue.py", line 113, in read_items
    for item in items:
TypeError: 'NoneType' object is not iterable


[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/syft/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/queue/zmq_queue.py", line 113, in read_items
    for item in items:
TypeError: 'NoneType' object is not iterable


[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK


In [ ]:
# args, _, _, values = inspect.getargvalues(process_all.local_function)


<function __main__.process_all(domain, x)>

TypeError: process_all() missing 1 required positional argument: 'domain'

Usecase 1: What if you have a syft function thats nested with jobs?

Subproblem 1.1: We need to code before submitting

- solution 1: remove the job aspect from local testing
- solution 2: you mimick the domain architecture locally
- solution 3: you run this on the low side on mock data


Subproblem 1.2: We need to be able to run this on the high side

Solution
- move requests / code / inputs from low side to high side
- run job on the high side
- get results from job and load them into the low side

Usecase 2: What if you have a syft function that takes days to run on the high side?

See solution for 1.2

Solutions based on example

- we have an optional mock sample in asset creation
- we split submission and code_request on the DS side
- DS can call code that is submitted

- what is the UX for running on the low side as a test vs a code request

ux for running as a test locally
my_function(asset.mock_sample)

ux for running as a test on the low side

ds_client.code.my_function(asset.mock)
-> Should print info: runs on mock data

ux for running the code on real data

ds_client.code.my_function(asset)
-> Should print info: runs on real data

the ux for a code request

ds_client.make_code_request(

ds_client.submit(code2)

In [17]:
ds_client.submit(code1)

AttributeError: 'DomainClient' object has no attribute 'submit'

In [ ]:
ds_client.request(code1)

In [ ]:
do_client.code

In [ ]:
request = do_client.requests[0]

code1:
    blbalblab
    code()

code2:
    blballbbla

In [ ]:
request.code2

In [ ]:
do_client.all_code.code2

When doing local execution **without** a domain

In [ ]:
my_syft_function(x, y)

When doing local execution **with** a domain

- is it relevant to think about whether we need to configure the domain (e.g. consumsers)
- do we create the domain explicitly or does it happen on the fly?
- how do we indicate in the function call that we want to create a domain?

When doing **high side** execution **without** a domain

In [ ]:
users_function = func.unsafe_function
real_result = users_function(trade_data=pvt_data)

When doing **high side** exeuction **with** a domain

In [11]:
job = ds_client.code.process_all(x=x_ptr, blocking=False)
sleep(5)

16/11/23 19:29:10 FUNCTION LOG (6470a06b282445ab8b912302c8c33a52): Launching jobs
16/11/23 19:29:10 FUNCTION LOG (6470a06b282445ab8b912302c8c33a52): starting aggregation
16/11/23 19:29:10 FUNCTION LOG (6470a06b282445ab8b912302c8c33a52): Done
16/11/23 19:29:11 FUNCTION LOG (511d17d79d0b450ab0f4d410bbbbe688): starting batch 1
16/11/23 19:29:11 EXCEPTION LOG (511d17d79d0b450ab0f4d410bbbbe688):

Encountered while executing process_batch:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1087, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 10, in user_func_process_batch_563f3f1899d092d0501158fbb086f3fd2c79a010c4e85bfe4fe12e0e1c01abde_3fa7f836b5a65455ef117b87f10f6a3f711d956132679e32522b6ae4c1440ec5
  File "<string>", line 6, in process_batch
IndexError: list index out of range

    3      print(f'starting batch {batch}')

In [12]:
job.subjobs[0].logs()

starting batch 1

Exception encountered while running process_batch, please contact the Node Admin for more info.


In [13]:
job.subjobs

[syft.service.job.job_stash.Job, syft.service.job.job_stash.Job]

In [14]:
client.jobs[0].subjobs[0].logs()

starting batch 1


Encountered while executing process_batch:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1087, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 10, in user_func_process_batch_563f3f1899d092d0501158fbb086f3fd2c79a010c4e85bfe4fe12e0e1c01abde_3fa7f836b5a65455ef117b87f10f6a3f711d956132679e32522b6ae4c1440ec5
  File "<string>", line 6, in process_batch
IndexError: list index out of range

    3      print(f'starting batch {batch}')
    4      from time import sleep
--> 5      [1, 2, 3, 4][5]
    6      sleep(1)


In [23]:
job.subjobs

[syft.service.job.job_stash.Job, syft.service.job.job_stash.Job]

In [12]:
job.subjobs[0].logs(stderr=True)

IndexError: list index out of range

starting batch 2


Encountered while executing process_batch:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1072, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 10, in user_func_process_batch_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_3fa7f836b5a65455ef117b87f10f6a3f711d956132679e32522b6ae4c1440ec5
  File "<string>", line 6, in process_batch
IndexError: list index out of range

    3      print(f'starting batch {batch}')
    4      from time import sleep
--> 5      [1, 2, 3, 4][5]
    6      sleep(1)


In [10]:
job.wait()

Pointer:
None

In [11]:
sum([j.wait().get() for j in job.subjobs])

TypeError: unsupported operand type(s) for +: 'int' and 'Err'

In [12]:
# import sys

In [10]:
# print("D")
# print("A")

# try:
#     raise ValueError()
# except Exception as e:
#     tb = e.__traceback__
# #     print(len([tb.tb_next while tb is not None])
# #     trace = []
# #     while tb is not None:
# #         trace.append({
# #             "filename": tb.tb_frame.f_code.co_filename,
# #             "name": tb.tb_frame.f_code.co_name,
# #             "lineno": tb.tb_lineno
# #         })
# #         tb = tb.tb_next


# #     print(sys.exc_info()[-1].tb_lineno)

In [11]:
# n = 0
# while tb is not None:
#     tb = tb.tb_next
#     n+=1

In [12]:
# n

In [13]:
# trace

In [14]:
import traceback
import sys

In [15]:
# try:
#     raise ValueError()
# except Exception as e:
#     print(sys.exception())
# #     print()


# # #     traceback.print_stack()


# # #     tb = e.__traceback__
# # #     traceback.print_tb(type(e), e, tb)

In [16]:
# print("\n".join(['def user_func_process_batch_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_bd42b6607712b09a0b187e3c67030f772c5dcd3c4152e4abebcb3a9f0a8259e4(batch):',
#            '', '    def process_batch(batch):', "        print(f'starting batch {batch}')",
#            '        from time import sleep', '        sleep(1)', '        [1, 2, 3][5]',
#            "        print('done')", '        return batch + 1', '    result = process_batch(batch=batch)',
#            '    return result']))